In [ ]:
import os, sys, csv
import importlib
import json, re

#os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.append(os.path.join('../'))

import pickle
import librosa.display
import scipy
import numpy as np
import matplotlib.pyplot as plt
import ipympl
import ipywidgets as wdg
from ipywidgets import Layout, HBox, VBox
import pygame

import torchaudio
import torch
import IPython.display as ipd

import src.recog as recog
import src.utils as utils

sys.path.append(os.path.join('../ast/src'))
from models import ASTModel

os.environ['TORCH_HOME'] = '../ast/pretrained_models'

torchaudio.set_audio_backend("soundfile")

In [ ]:
sample_freq = 16000
mel_bins = 128
target_length = 1024

# Load a flac file from 0(s) to 60(s) and resample to 4.41 KHz
filename = "../media/2022-3-5_TA.flac"
filename = "../media/2022-3-5_TA_last4.flac"
filename = "../media/2022-3-10_Rin_noon_6min.flac"


interval = 10e-3 #ms
win_length = 25e-3 #ms

# hop_length is number of samples between successive frames.
hop_length=int(sample_freq * interval)


#ast_model = None
if not "ast_model" in vars(sys.modules[__name__]) or not ast_model:
    ast_model = recog.Recog(ASTModel, target_length, "../ast/egs/audioset/data/class_labels_indices.csv")

In [ ]:
onto = utils.Ontology('../ontology/ontology.json')
interests = onto[utils.reg(r"^(Singing|Music)$")]

In [ ]:
# load
with open("recog_test.pickle", mode="rb") as f:
    y, sr = pickle.load(f)

In [ ]:
#y, sr = librosa.load(filename, sr=sample_freq, mono=False)
#y, sr = librosa.load(filename, sr=sample_freq, mono=False, offset=6*60+15, duration=1*60+30)
y, sr = librosa.load(filename, sr=44100, mono=False, offset=295, duration=2*60+30)

In [ ]:
# save
with open("recog_test.pickle", mode="wb") as f:
    pickle.dump((y, sr), f)

In [ ]:
importlib.reload(recog)

%matplotlib widget
fig, ax = plt.subplots(nrows=1, sharex=True)
recog.Recog.waveplot(plt, librosa, y, sr, ax=ax, max_sr=2, )
#fbank_orig = recog.Recog.fbank(y, sr, target_length*2, mel_bins=mel_bins, frame_shift=10)
#recog.Recog.specshow(fig, librosa, fbank_orig, sr, hop_length, title="Mel spectrogram", cb=False, w=16, h=4, ax=ax[1])
fig.set_size_inches(12, 3)

print(sr, y.shape, type(y))


# for stop, pygame.mixer.quit()
pygame.mixer.pre_init(sr, size=-16, channels=2)
pygame.mixer.init()

sound = pygame.sndarray.make_sound((32768*y.transpose(1,0).copy(order="C")).astype(np.int16))

start_b = wdg.Button(description='Start')
stop_b = wdg.Button(description="Stop")
start_b.on_click(lambda b: sound.play() )
stop_b.on_click(lambda b:  sound.stop() )

HBox([start_b, stop_b])

In [ ]:
delta = .5
duration = 2
start = 0#+100
cut_length = 300#60+30
infer_series = utils.if_not_defined(__name__, "infer_series", {})
wav = y[..., int(sr*start):int(sr*(cut_length+start))]
entire_abs_mean = 0.04501902312040329#np.abs(y).mean()

In [ ]:
importlib.reload(recog)
importlib.reload(utils)
detect_series, detect_d_series, conc_series, states = recog.Recog.detect_music(ast_model, infer_series, start, delta, duration, start, wav, sr, onto, interests, entire_abs_mean=entire_abs_mean,
                                                                              interval_plan=[])

In [ ]:
# https://qiita.com/syoyo/items/a43e568ee5309c4f17d3
importlib.reload(recog)

%matplotlib widget

tmp={}

def onclick(event):
    text = 'event.button=%d,  event.x=%d, event.y=%d, event.xdata=%f, event.ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
    print(text)
    tmp["y_cut"] = y[:, int(sr*event.xdata):-1]
    sound = pygame.sndarray.make_sound((32768*tmp["y_cut"].transpose(1,0).copy(order="C")).astype(np.int16))
    sound.play()
    tmp["sound"] = sound

def on_click_callback(clicked_button: wdg.Button) -> None:
    tmp["sound"].stop()

def plot_all():
    button = wdg.Button(description='Stop')
    qbutton = wdg.Button(description="Quit")
    button.on_click(on_click_callback)
    qbutton.on_click(lambda b: pygame.mixer.quit())

    # for stop, pygame.mixer.quit()
    pygame.mixer.pre_init(sr, size=-16, channels=2)
    pygame.mixer.init()

    fig, ax = plt.subplots(nrows=5, sharex=True)
    recog.Recog.waveplot(plt, librosa, wav, sr, ax=ax[0], offset=start, max_sr=2)
    recog.Recog.classifyshow(onto, infer_series, ax=ax[1])
    recog.Recog.state_show([recog.State.Music, recog.State.Talking, recog.State.Other], conc_series, ax=ax[2])
    recog.Recog.detect_show(detect_series, xstep=4, ax=ax[3])
    recog.Recog.detect_show(detect_d_series, xstep=int(cut_length//30), ax=ax[4])

    fig.set_size_inches(16, 8)
    for a in ax:
        a.xaxis.grid()
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    #display(txt)
    return HBox([button, qbutton])

plot_all()

In [ ]:
cut_length = 60+30+30
start = recog.Recog.last_cur_time(states["prev_c_results"])
wav = y[..., int(sr*start):int(sr*(cut_length+start))]

In [ ]:
print(states["cur_time"], wav.shape)

In [ ]:
for k in list(infer_series.keys()):
    if k[0] > 1*60+25*0:
        infer_series.pop(k)

In [ ]:
def state_renew():
    for k,v in states.items():
        if str(type(v)) == str(recog.State):
            for st1 in recog.State:
                for st2 in recog.State:
                    st = st1|st2
                    if v.value == st.value:
                        print(k, v.value, st, st.value)
                        states[k] = st

In [ ]:
importlib.reload(recog) # do not reload! Otherwise isinstance(states["prev_judge"], recog.State) == False
state_renew()
detect_series, detect_d_series, conc_series, _ = recog.Recog.detect_music(ast_model, infer_series, start, delta, duration, start, wav, sr, onto, interests, **states, entire_abs_mean=entire_abs_mean)

In [ ]:
%matplotlib widget

plot_all()

In [ ]:
importlib.reload(recog)
importlib.reload(utils)

tmp_series = utils.if_not_defined(__name__, "tmp_series", {})
recog.Recog.back_to_change(ast_model, recog.State.Music|recog.State.Start, infer_series, 40, 0.5, 2, 26, wav, sr, onto, interests, thres = 1.0)

#recog.Recog.is_music_starting(ast_model, infer_series, 30, 0.5, 2, 20.0, 5, wav, sr, onto, interests, thres=0.5)

In [ ]:
for ident, ratio in infer_series[(75.5,0.5)][0].items():
    print(f"{onto.to_name(ident)}: {ratio}")

In [ ]:
importlib.reload(recog)
recog.Recog.categorize(infer_series[(0.0, 0.5)][0], 35, 35, list(map(lambda o: o["id"], onto["^(Music|Singing)$"])), onto["^Human sounds$"][0]["id"])

In [ ]:
importlib.reload(recog)
estim_seg = recog.Recog.estim_segment(ast_model, onto, infer_series, 10, 0.5, 4,   wav, sr, onto, interests, music_thres = 35, hs_thres = 35)
print(estim_seg)

In [ ]:
importlib.reload(recog)

recog.Recog.judge_segment(ast_model, onto, {}, 1000, 0.5, 2, wav, sr, onto, interests, aft_thres=3)

In [ ]:
importlib.reload(utils)

# {(0,1): ["A"], (0,0.9): ["B"], (0.5, 0.3): ["C"]}
for itv, states in utils.each_state(estim_seg):
    print(f"{itv}: {states}")

In [ ]:
recog.State.Talking|recog.State.InProgress in (recog.State.Talking | recog.State.InProgress)